In [ ]:
# Install ctransformers and dependencies# Install dependencies (Colab-safe)
!pip install -q ctransformers langgraph langchain faiss-cpu sentence-transformers


In [ ]:
!pip install -U langchain-community

In [ ]:
from ctransformers import AutoModelForCausalLM
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal, List

# Load quantized LLM using ctransformers (fast and CPU-friendly)
llm = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
    model_file="mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    model_type="mistral",
    context_length=2048,
    gpu_layers=0  # CPU only
)

# Documents: You can replace with your own domain-specific data
docs = [
    "User A enjoys science fiction and prefers short books.",
    "Book: Dune – A classic sci-fi epic.",
    "Book: Project Hail Mary – A fast-paced science fiction novel.",
    "Book: The Hobbit – A short fantasy story with adventure.",
    "Book: Foundation – A legendary sci-fi series by Isaac Asimov.",
    "User B enjoys romance and character-driven narratives.",
    "Book: Pride and Prejudice – A romantic classic.",
    "Book: The Notebook – A deeply emotional romance.",
]

# Vector store (RAG backend)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
faiss_docs = splitter.create_documents(docs)
vectorstore = FAISS.from_documents(faiss_docs, embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# LangGraph Agent State
class AgentState(TypedDict):
    user_query: str
    retrieved_docs: List[str]
    final_response: str
    next_action: Literal["retrieve", "reason", "end"]

# Planner Node (decides what to do)
def planner_node(state: AgentState) -> AgentState:
    if not state.get("retrieved_docs"):
        next_action = "retrieve"
    elif state.get("final_response"):  # Already has a response
        next_action = "end"
    else:
        next_action = "reason"
    return {**state, "next_action": next_action}


# Retrieval Tool
def retrieval_node(state: AgentState) -> AgentState:
    docs = retriever.get_relevant_documents(state["user_query"])
    return {
        **state,
        "retrieved_docs": [doc.page_content for doc in docs],
        "next_action": "reason"
    }

# Reasoning Tool (LLM)
def reasoning_node(state: AgentState) -> AgentState:
    context = "\n".join(state["retrieved_docs"])
    prompt = f"""You are a recommendation system.

Only use the information from the context below to answer the user. Do not use any external knowledge.

User Query:
{state['user_query']}

Context:
{context}

Answer with a personalized recommendation using only the above context.
"""
    response = llm(prompt)
    return {
        **state,
        "final_response": response.strip(),
        "next_action": "end"
    }

# LangGraph Agentic Flow
builder = StateGraph(AgentState)
builder.add_node("planner", planner_node)
builder.add_node("retrieve", retrieval_node)
builder.add_node("reason", reasoning_node)

builder.set_entry_point("planner")

builder.add_conditional_edges("planner", lambda s: s["next_action"], {
    "retrieve": "retrieve",
    "reason": "reason",
    "end": END
})

builder.add_edge("retrieve", "planner")
builder.add_edge("reason", "planner")

graph = builder.compile()

# Run the agent
query = "Can you suggest a fast sci-fi book for someone who likes short stories?"
result = graph.invoke({"user_query": query})

print("🤖 Final Answer:")
print(result["final_response"])
